# OSM to GeoJSON

Transform an OSM (XML) file into a pair of .geojson files, for easier drawing on a map.

We produce one .geojson file with all ways from the original OSM data, and a second one with just the ways that are tagged as "cycleways".

Depends on OSM (XML) files already having been extracted for the locality, see OSM_Filter.ipynb first.

In [1]:
# Specify the locality whose OSM file we want to load

#locality = 'Mount Eliza'
#locality = 'Frankston'
locality = 'Langwarrin'

In [2]:
import json
import math
import os
import sys
import xml.dom.minidom

from geographiclib.geodesic import Geodesic

geod = Geodesic.WGS84  # define the WGS84 ellipsoid

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)

In [3]:
# Load the XML file into memory
# This assumes that we have reduced the OpenStreetMap data down to a small enough locality
# that the in-memory approach is feasible

osm_filename = os.path.join(os.pardir, 'data_sources', 'Locality_' + locality.replace(' ', '_') + '.osm')

doc = xml.dom.minidom.parse(osm_filename)

In [4]:
# Get ways and nodes from XML document
ways_xml  = doc.getElementsByTagName('way')
nodes_xml = doc.getElementsByTagName('node')

# Cache ways in dict by name
ways = {}
for way in ways_xml:
    tags = way.getElementsByTagName('tag')
    for tag in tags:
        k = tag.getAttribute('k')
        if k == 'name':
            v = tag.getAttribute('v').upper()
            
            if v in ways:
                ways[v].append(way)
            else:
                ways[v] = [way]
                
# Cache nodes in dict by id/ref
nodes = {}
for node in nodes_xml:
    id = node.getAttribute('id').upper()
    nodes[id] = node
    
print('Way count:  %d' % ways_xml.length)
print('Node count: %d' % nodes_xml.length)

Way count:  7539
Node count: 44976


In [5]:
def find_way_by_name(name):
    return ways[name.upper()]

In [6]:
#x = find_way_by_name('Millbank Drive')

In [7]:
def print_way(way):
    tags = way.getElementsByTagName('tag')
    for tag in tags:
        k = tag.getAttribute('k')
        if k == 'name':
            v = tag.getAttribute('v')
            print(v)
            return

In [8]:
#print_way(x[0])

In [9]:
def find_ways_by_attribute(attr, exclusion_val_list=['no']):
    matching_ways = []
    
    for way in ways_xml:
        tags = way.getElementsByTagName('tag')
        for tag in tags:
            k = tag.getAttribute('k')
            if k.upper().startswith(attr.upper()):
                v = tag.getAttribute('v')
                for exclusion_val in exclusion_val_list:
                    if v == exclusion_val:
                        k = ''
            if k == attr:
                matching_ways.append(way)
    
    return matching_ways

In [10]:
cycleways = find_ways_by_attribute('cycleway')

In [11]:
print('Cycleway count:  %d' % len(cycleways))

Cycleway count:  76


In [12]:
for cycleway in cycleways:
    print_way(cycleway)

Western Port Highway
Cranbourne - Frankston Road
Cranbourne Road
Western Port Highway
Western Port Highway
Western Port Highway
Western Port Highway
Western Port Highway
Western Port Highway
Cranbourne Road
Cranbourne Road
Cranbourne Road
Peninsula Link Trail
Peninsula Link Trail
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Peninsula Link Trail
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Western Port Highway
Western Port Highway
Western Port Highway
Western Port Highway
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Western Port Highway
Cranbourne Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne - Frankston Road
Cranbourne Road
C

In [13]:
def way_to_feature(way):
    # Create LineString list of coordinates from nodes attached to way
    coordinates = []
    
    way_node_refs = way.getElementsByTagName('nd')
    
    for way_node_ref in way_node_refs:
        ref = way_node_ref.getAttribute('ref').upper()
        way_node = nodes[ref]
        # lat -38 lon 145
        way_node_lat = way_node.getAttribute('lat')
        way_node_lon = way_node.getAttribute('lon')
        coordinates.append([way_node_lon, way_node_lat])
        
    geometry = {
        'type': 'LineString',
        'coordinates': coordinates
    }
    
    # Fill in properties for the way
    properties = {}
    
    properties['id'] = way.getAttribute('id').upper()
    properties['version'] = way.getAttribute('version').upper()
    properties['timestamp'] = way.getAttribute('timestamp')
    
    tags = way.getElementsByTagName('tag')
    for tag in tags:
        k = tag.getAttribute('k')
        v = tag.getAttribute('v')
        properties[k] = v
    
    # Build feature from geometry and properties
    feature = {
        'type':       'Feature',
        'properties': properties,
        'geometry':   geometry
    }
    
    return feature

In [14]:
def ways_to_featurecollection(ways):
    features = []
    
    for way in ways:
        features.append(way_to_feature(way))
        
    featurecollection = {
        'type':     'FeatureCollection',
        'features': features
    }
    
    return featurecollection

In [15]:
fc = ways_to_featurecollection(cycleways)

output_filename = os.path.join(os.pardir, 'data_sources', 'Locality_OSM_cycleway_' + locality.replace(' ', '_') + '.geojson')
print(output_filename)

with open(output_filename, 'w') as outfile:
    json.dump(fc, outfile, indent=4)
    
    outfile.close()

..\data_sources\Locality_OSM_cycleway_Langwarrin.geojson


In [16]:
fc = ways_to_featurecollection(ways_xml)

output_filename = os.path.join(os.pardir, 'data_sources', 'Locality_OSM_all_' + locality.replace(' ', '_') + '.geojson')
print(output_filename)

with open(output_filename, 'w') as outfile:
    json.dump(fc, outfile, indent=4)
    
    outfile.close()

..\data_sources\Locality_OSM_all_Langwarrin.geojson
